In [1]:
import feedparser
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3")
conlt = engine.connect()

clean_path = "../data/clean/"
raw_path = "../data/raw/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
pdf_path = "../PDF/"

pd.set_option('display.max_colwidth', 255)
pd.set_option('display.max_rows',None)
url = "https://feeds.feedburner.com/Setorth-form45-en"

today = date.today()
year = 2022
mmdd_str = today.strftime('%m%d')
mmdd_str

'1110'

In [2]:
#today = date(2022, 11, 4)
mmdd_str = today.strftime('%m%d')
mmdd_str

'1110'

In [38]:
rss_source = feedparser.parse(url)
f45_number = len(rss_source.entries)
print("Number of F45: ", f45_number)

Number of F45:  166


In [39]:
f45_items = []

for x in range(f45_number):
    f45_content = rss_source.entries[x]
    f45_item = {}
    
    print("\n----------------------------------\n")
    
    print("F45: " + str(x))
    title_ary = f45_content.title.partition(' ')
    f45_item['name'] = title_ary[0].strip() 
    print("Title: ", f45_item['name'])  
    f45_item['year'] = year
    print("Year: ", f45_item['year'])      
    qtr_ary = title_ary[2].partition(' (F45)')
    f45_item['quarter'] = qtr_ary[0][-1]    
    print("Quarter: ", f45_item['quarter'])    
    f45_item['link'] = f45_content.link
    print("Link: ", f45_item['link'])
    f45_item['published'] = f45_content.published
    print("Published: ", f45_item['published'])  
    f45_items.append(f45_item)


----------------------------------

F45: 0
Title:  D
Year:  2022
Quarter:  3
Link:  https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355834120
Published:  Thu, 10 Nov 2022 21:56:06 +0700

----------------------------------

F45: 1
Title:  SORKON
Year:  2022
Quarter:  3
Link:  https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355833920
Published:  Thu, 10 Nov 2022 21:53:21 +0700

----------------------------------

F45: 2
Title:  TPA
Year:  2022
Quarter:  3
Link:  https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355833380
Published:  Thu, 10 Nov 2022 21:44:51 +0700

----------------------------------

F45: 3
Title:  BSM
Year:  2022
Quarter:  3
Link:  https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355833140
Published:  Thu, 10 Nov 2022 21:38:42 +0700

----------------------------------

F45: 4
Title:  BE8
Year:  2022
Quarter:  3
Link:  https://classic.set.or.th/se

In [40]:
df = pd.DataFrame(f45_items)
df[['name','year','quarter','published']]

,name,year,quarter,published
0,D,2022,3,"Thu, 10 Nov 2022 21:56:06 +0700"
1,SORKON,2022,3,"Thu, 10 Nov 2022 21:53:21 +0700"
2,TPA,2022,3,"Thu, 10 Nov 2022 21:44:51 +0700"
3,BSM,2022,3,"Thu, 10 Nov 2022 21:38:42 +0700"
4,BE8,2022,3,"Thu, 10 Nov 2022 21:38:24 +0700"
5,TEAM,2022,3,"Thu, 10 Nov 2022 21:35:23 +0700"
6,TPAC,2022,3,"Thu, 10 Nov 2022 21:15:32 +0700"
7,PREB,2022,3,"Thu, 10 Nov 2022 21:04:10 +0700"
8,EFORL,2022,3,"Thu, 10 Nov 2022 20:35:23 +0700"
9,HEMP,2022,3,"Thu, 10 Nov 2022 20:33:56 +0700"


In [41]:
df.loc[(df.quarter == 'y') ,['year','quarter']] = ['2021','4']
df.groupby(['year','quarter']).count()

name  link  published
year quarter                       
2022 1           2     2          2
     2           3     3          3
     3         161   161        161

In [42]:
df.loc[(df.quarter == 'y') ,['year','quarter']] = ['2021','4']
df.groupby(['year','quarter']).count()

name  link  published
year quarter                       
2022 1           2     2          2
     2           3     3          3
     3         161   161        161

In [8]:
df.loc[(df.name == 'COTTO') ,['year','quarter']] = ['2022','3']
df.groupby(['year','quarter']).count()

name  link  published
year quarter                       
2022 1           2     2          2
     2           3     3          3
     3         154   154        154

### After change all illogical quarters

In [43]:
df.quarter = df.quarter.astype(int)
df.groupby(['year','quarter']).count()

name  link  published
year quarter                       
2022 1           2     2          2
     2           3     3          3
     3         161   161        161

In [44]:
### First equals to latest published pdf file
df = df.drop_duplicates(subset='name', keep='first')
df.shape

(157, 5)

In [45]:
file_name = 'F45-RAW-' + mmdd_str + '.csv'
raw_file = raw_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df[['name','year','quarter','published']].to_csv(output_file, header=True, index=False, sep=',')
df[['name','year','quarter','published']].to_csv(box_file,    header=True, index=False, sep=',')
df[['name','year','quarter','published']].to_csv(one_file,    header=True, index=False, sep=',')
df[['name','year','quarter','published']].to_csv(raw_file,    header=True, index=False, sep=',')

In [46]:
sql = '''
SELECT *
FROM exempts
ORDER BY name'''
df_exempts = pd.read_sql(sql, conlt)
df_exempts.shape[0]

403

### Delete from exempts table

In [47]:
name = 'SABUY'
sqlDel = """
DELETE FROM exempts
WHERE name = '%s' """
sqlDel = sqlDel % name
print(sqlDel)


DELETE FROM exempts
WHERE name = 'SABUY' 


In [48]:
rp = conlt.execute(sqlDel)
rp.rowcount

0

### End of delete from exempts table

In [49]:
df_merge = pd.merge(df, df_exempts, on='name', how='outer', indicator=True)
df_merge.shape

(489, 7)

### Tickers that are in Exempts table

In [50]:
in_exempts = df_merge.loc[
    df_merge['_merge'] == 'both',
    ['name','year','quarter','published','link']
    
]
in_exempts.year = in_exempts.year.astype(int)
in_exempts.quarter = in_exempts.quarter.astype(int)
in_exempts.sort_values(by=['published'],ascending=[False])

,name,year,quarter,published,link
1,SORKON,2022,3,"Thu, 10 Nov 2022 21:53:21 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355833920
3,BSM,2022,3,"Thu, 10 Nov 2022 21:38:42 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355833140
5,TEAM,2022,3,"Thu, 10 Nov 2022 21:35:23 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355832810
10,AMC,2022,3,"Thu, 10 Nov 2022 20:29:12 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355829360
11,NTV,2022,3,"Thu, 10 Nov 2022 20:25:59 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355828990
13,WAVE,2022,3,"Thu, 10 Nov 2022 20:22:39 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355828580
16,DOD,2022,3,"Thu, 10 Nov 2022 20:04:47 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355826740
17,NYT,2022,3,"Thu, 10 Nov 2022 20:04:16 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355826700
22,CRANE,2022,3,"Thu, 10 Nov 2022 19:39:43 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355824480
23,TCMC,2022,3,"Thu, 10 Nov 2022 19:33:39 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355824160


In [51]:
in_exempts.sort_values(by=['published'],ascending=[False]).shape[0]

71

### Not in exempts table

In [52]:
df_out = df_merge.loc[
    df_merge['_merge'] == 'left_only',
    ['name','year','quarter','published','link']
]
df_out.year = df_out.year.astype(int)
df_out.quarter = df_out.quarter.astype(int)
df_out.sort_values(by=['published'],ascending=[False])

,name,year,quarter,published,link
0,D,2022,3,"Thu, 10 Nov 2022 21:56:06 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355834120
2,TPA,2022,3,"Thu, 10 Nov 2022 21:44:51 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355833380
4,BE8,2022,3,"Thu, 10 Nov 2022 21:38:24 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355833120
6,TPAC,2022,3,"Thu, 10 Nov 2022 21:15:32 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355831360
7,PREB,2022,3,"Thu, 10 Nov 2022 21:04:10 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355830730
8,EFORL,2022,3,"Thu, 10 Nov 2022 20:35:23 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355829710
9,HEMP,2022,3,"Thu, 10 Nov 2022 20:33:56 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355829650
12,MBAX,2022,3,"Thu, 10 Nov 2022 20:25:43 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355828970
14,SAK,2022,3,"Thu, 10 Nov 2022 20:19:17 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355828360
15,BANPU,2022,3,"Thu, 10 Nov 2022 20:07:15 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355827200


In [53]:
#df_out = df_out.drop(df_out.index[df_out['name'] == "SCC"])
df_out.shape[0]

86

In [54]:
sql = '''
SELECT *
FROM tickers
ORDER BY name'''
df_tickers = pd.read_sql(sql, conlt)
df_tickers.shape

(401, 9)

In [55]:
df_merge2 = pd.merge(df_out, df_tickers, on='name', how='outer', indicator=True)
df_merge2.shape

(442, 14)

### There are no ticker records

In [56]:
df_merge2.loc[
    df_merge2['_merge'] == 'left_only',
    ['name','year','quarter','published','link']
]

,name,year,quarter,published,link
0,D,2022.0,3.0,"Thu, 10 Nov 2022 21:56:06 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355834120
1,TPA,2022.0,3.0,"Thu, 10 Nov 2022 21:44:51 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355833380
3,TPAC,2022.0,3.0,"Thu, 10 Nov 2022 21:15:32 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355831360
5,EFORL,2022.0,3.0,"Thu, 10 Nov 2022 20:35:23 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355829710
6,HEMP,2022.0,3.0,"Thu, 10 Nov 2022 20:33:56 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355829650
8,SAK,2022.0,3.0,"Thu, 10 Nov 2022 20:19:17 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355828360
10,PEACE,2022.0,3.0,"Thu, 10 Nov 2022 20:01:00 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355826370
11,SICT,2022.0,3.0,"Thu, 10 Nov 2022 20:00:47 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355826340
13,BBIK,2022.0,3.0,"Thu, 10 Nov 2022 19:47:29 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355825150
15,TPS,2022.0,3.0,"Thu, 10 Nov 2022 19:21:35 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355823290


In [57]:
df_merge2.loc[
    df_merge2['_merge'] == 'left_only',
    ['name','year','quarter','published','link','id','market']
].shape

(41, 7)

### There are ticker records

In [58]:
df_out2 = df_merge2.loc[
    df_merge2['_merge'] == 'both',
    ['name','year','quarter','published','link','id','market']
]
df_out2

,name,year,quarter,published,link,id,market
2,BE8,2022.0,3.0,"Thu, 10 Nov 2022 21:38:24 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355833120,745.0,mai
4,PREB,2022.0,3.0,"Thu, 10 Nov 2022 21:04:10 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355830730,372.0,SET
7,MBAX,2022.0,3.0,"Thu, 10 Nov 2022 20:25:43 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355828970,288.0,mai
9,BANPU,2022.0,3.0,"Thu, 10 Nov 2022 20:07:15 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355827200,47.0,SET50 / SETCLMV / SETTHSI
12,BPP,2022.0,3.0,"Thu, 10 Nov 2022 19:50:20 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355825740,74.0,SET / SETCLMV / SETTHSI
14,BBL,2022.0,3.0,"Thu, 10 Nov 2022 19:23:41 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355823500,50.0,SET50 / SETHD / SETTHSI
16,S11,2022.0,3.0,"Thu, 10 Nov 2022 19:12:29 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355822890,412.0,sSET
18,SYNEX,2022.0,3.0,"Thu, 10 Nov 2022 18:48:32 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355819940,495.0,SET100 / SETTHSI
21,EPG,2022.0,2.0,"Thu, 10 Nov 2022 18:37:29 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355818650,162.0,SET100 / SETTHSI
22,AIT,2022.0,3.0,"Thu, 10 Nov 2022 18:30:40 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355817530,11.0,sSET


In [59]:
df_out2 = df_out2[df_out2.year.notnull()]
df_out2.shape

(45, 7)

In [60]:
df_out2['year'] = df_out2['year'].astype(int)
df_out2['quarter'] = df_out2['quarter'].astype(int)
df_out2.shape

(45, 7)

In [61]:
file_name = 'F45-CLEAN-' + mmdd_str + '.csv'
clean_file = clean_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_out2[['name','year','quarter','published','market']].sort_values(['published'],ascending=[False]).to_csv(output_file, header=True, index=False, sep=',')
df_out2[['name','year','quarter','published','market']].sort_values(['published'],ascending=[False]).to_csv(clean_file, header=True, index=False, sep=',')
df_out2[['name','year','quarter','published','market']].sort_values(['published'],ascending=[False]).to_csv(box_file, header=True, index=False, sep=',')
df_out2[['name','year','quarter','published','market']].sort_values(['published'],ascending=[False]).to_csv(one_file, header=True, index=False, sep=',')

In [62]:
sql = '''
SELECT * 
FROM epss
WHERE year = 2022'''
df_epss = pd.read_sql(sql, conlt)
df_epss.shape

(584, 14)

In [63]:
df_merge3 = pd.merge(df_out2, df_epss, on=['name','year','quarter'], how='outer', indicator=True)
df_merge3.shape

(587, 19)

### Already input, display profit amt & eps to check with new F45

In [64]:
df_merge3[df_merge3['_merge'] == 'both']

,name,year,quarter,published,link,id_x,market,id_y,q_amt,y_amt,aq_amt,ay_amt,q_eps,y_eps,aq_eps,ay_eps,ticker_id,publish_date,_merge
1,PREB,2022,3,"Thu, 10 Nov 2022 21:04:10 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355830730,372.0,SET,22047.0,114789.0,36515.0,326832.0,155599.0,0.3720,0.1180,1.0590,0.5040,372.0,2022-11-10,both
2,MBAX,2022,3,"Thu, 10 Nov 2022 20:25:43 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355828970,288.0,mai,22045.0,68490.0,35634.0,128584.0,25051.0,0.3600,0.1900,0.6700,0.1300,288.0,2022-11-10,both
3,BANPU,2022,3,"Thu, 10 Nov 2022 20:07:15 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355827200,47.0,SET50 / SETCLMV / SETTHSI,22035.0,17743984.0,3505036.0,40797233.0,6365597.0,2.6220,0.6830,6.0300,1.2500,47.0,2022-11-10,both
4,BPP,2022,3,"Thu, 10 Nov 2022 19:50:20 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355825740,74.0,SET / SETCLMV / SETTHSI,22036.0,2315177.0,596484.0,5918742.0,2756268.0,0.7600,0.1950,1.9420,0.9040,74.0,2022-11-10,both
5,BBL,2022,3,"Thu, 10 Nov 2022 19:23:41 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355823500,50.0,SET50 / SETHD / SETTHSI,21906.0,7656990.0,6909204.0,21736126.0,20189067.0,4.0100,3.6200,11.3900,10.5800,50.0,2022-10-20,both
6,S11,2022,3,"Thu, 10 Nov 2022 19:12:29 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355822890,412.0,sSET,22049.0,123390.0,108081.0,265343.0,301346.0,0.2000,0.1800,0.4300,0.4900,412.0,2022-11-10,both
7,SYNEX,2022,3,"Thu, 10 Nov 2022 18:48:32 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355819940,495.0,SET100 / SETTHSI,22051.0,215475.0,194476.0,630002.0,590743.0,0.2500,0.2300,0.7400,0.7000,495.0,2022-11-10,both
9,AIT,2022,3,"Thu, 10 Nov 2022 18:30:40 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355817530,11.0,sSET,22031.0,142739.0,117156.0,430526.0,375429.0,0.1400,0.1100,0.4200,0.3600,11.0,2022-11-10,both
10,JMART,2022,3,"Thu, 10 Nov 2022 18:09:24 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355814160,236.0,SET50,22039.0,563490.0,269927.0,1278017.0,835638.0,0.3950,0.2610,0.9040,0.8380,236.0,2022-11-10,both
11,BAM,2022,3,"Thu, 10 Nov 2022 18:06:26 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355813220,709.0,SET100,22034.0,715833.0,576326.0,1858296.0,1613594.0,0.2200,0.1800,0.5700,0.5000,709.0,2022-11-10,both


In [65]:
df_merge3[df_merge3['_merge'] == 'both'].shape

(42, 19)

### New F-45

In [66]:
df_inp2 = df_merge3[df_merge3['_merge'] == 'left_only']
df_inp3 = df_inp2.copy()
df_inp3.shape

(3, 19)

In [67]:
df_inp3['year'] = df_inp3['year'].astype(str)
df_inp3['quarter'] = df_inp3['quarter'].astype(str)
final = df_inp3.sort_values('name',ascending=True)
final_str = final.name+' '+final.year+' '+final.quarter+' '+final.link
final_str

0        BE8 2022 3 https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355833120
8        EPG 2022 2 https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355818650
19    IMPACT 2022 2 https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355804290
dtype: object

In [68]:
df_inp3_q3 = df_inp3[df_inp3['quarter'] == '3']
final_q3 = df_inp3_q3.sort_values('name',ascending=True)
final_str_q3 = final_q3.name+' '+final_q3.year+' '+final_q3.quarter+' '+final_q3.link
final_str_q3

0    BE8 2022 3 https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355833120
dtype: object

In [69]:
df_inp3_q1 = df_inp3[df_inp3['quarter'] != '3']
final_q1 = df_inp3_q1.sort_values('name',ascending=True)
final_str_q1 = final_q1.name+' '+final_q1.year+' '+final_q1.quarter+' '+final_q1.link
final_str_q1

8        EPG 2022 2 https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355818650
19    IMPACT 2022 2 https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16680355804290
dtype: object